In [2]:
import numpy as np
from utilities import remove_cols
import json
import time
from src.learner import *

samples = np.genfromtxt('data/samples.csv', delimiter=",")

The final hyperparameters we settled on were:

Learning rate: 0.24
Batch size: 256
Epochs: 40

These produced a training and testing accuracy of 0.983 and 0.976, respectively, and were associated with a runtime of under one second. While slightly higher train and test accuracies could have been achieved with more advanced tuning and training (e.g. training on more epochs) this configuration allowed us to achieve near-perfect performance on the training and holdout sets and be able to train within reasonable time horizons.

In [3]:
with open('data/params.json', 'r') as f:
    cfg = json.load(f)

In [5]:
import pandas as pd
# inputs and outputs
X = remove_cols(np.genfromtxt('data/orth.csv', delimiter=","))
Y = remove_cols(np.genfromtxt('data/phon.csv', delimiter=","))
words = pd.read_csv('data/words.csv', header=None)
words = words[0].tolist()


We will start with 100 hidden units because of prior literature and because this is the value we tuned over.

In [6]:
hidden = 100

Before we run samples at the full 10K scale, let's run a number of longitudinal models, manipulating the random seed and writing off results periodically throughout training.

In [7]:
# the interval representing the number of epochs after which we write train and test data
iter_step = 1

In [8]:
# same sample as with tuning
np.random.seed(982)

train_n = X.shape[0]
test_n = train_n-600

sample = np.full(train_n, False, dtype=bool)

indices = np.random.choice(train_n, test_n, replace=False)

# Set chosen indices to True
sample[indices] = True

In [9]:
train_indices = np.where(sample)[0]
test_indices = np.where(~sample)[0]

In [24]:
model = learner(X, Y, 657, cfg['learning_rate'], hidden)
model.fit(X[sample], Y[sample], epochs=1000, batch_size=cfg['batch_size'], verbose=True)

Epoch 1/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.4155 - right_side_accuracy: 0.7838 - accuracy: 0.0557 - mse: 0.1706
Epoch 2/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.5344 - right_side_accuracy: 0.9120 - accuracy: 0.1021 - mse: 0.0769
Epoch 3/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.2720 - right_side_accuracy: 0.9531 - accuracy: 0.3608 - mse: 0.0407
Epoch 4/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.1453 - right_side_accuracy: 0.9680 - accuracy: 0.5072 - mse: 0.0263
Epoch 5/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.0925 - right_side_accuracy: 0.9739 - accuracy: 0.4897 - mse: 0.0204
Epoch 6/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.0718 - right_side_accuracy: 0.9761 - accuracy: 0.5664 - mse: 0.0178
Epoch 7/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.0613 - right_side_accuracy: 0.9774 - accuracy: 0.5840 - mse: 0.0166
Epoch 

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               10300     
                                                                 
 dense_1 (Dense)             (None, 66)                6666      
                                                                 
Total params: 16966 (66.27 KB)
Trainable params: 16966 (66.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:

def right_side_accuracy_batch(Y, Y_hat, theta=0.5, dichotomous=True):
    """
    Calculates accuracy for each example quickly.
    
    Parameters
    ----------
    Y: numpy.ndarray
        The targets.
    Y_hat: numpy.ndarray
        These are the predicted values.
    theta: float
        This value is the threshold to decide binary accuracy per unit on the output layer.
    dichotomous: bool
        Should the accuracies be dichotomous (default) or continuous (as the mean over units)
    
    Returns
    -------
    numpy.ndarray
        The accuracy for each example.
    
    
    """
    # Threshold predictions
    preds = (Y_hat > theta).astype(int)
    
    if dichotomous:
        accuracies = np.all(preds == Y, axis=1)
    if not dichotomous:
        accuracies = np.mean(preds == Y, axis=1)

    return accuracies


In [25]:
accuracies = right_side_accuracy_batch(Y, model.predict(X))

91/91 [==============================] - 0s 2ms/step


In [27]:
accuracies

array([False,  True, False, ..., False, False,  True])

In [17]:
len([acc for acc in accuracies if acc == 1])

1084

In [10]:
with open('outputs/hu100_longitudinal_test.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{}\n".format("word",
                                            "condition",
                                             "epoch",
                                             "loss",
                                             "accuracy",
                                             "mse",
                                             "learning_rate",
                                             "batch_size",
                                             "seed",
                                             "time"))
    for seed in range(1): 
        model = learner(seed, cfg['learning_rate'], hidden)
                
        start_time = time.time()

        for epoch in range(1): # 

            model.fit(X[sample], Y[sample], epochs=iter_step, batch_size=cfg['batch_size'], verbose=False)

            end_time = time.time()
            runtime = end_time - start_time

            loss_train, accuracy_train, __, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) # loss is duplicated so the 2nd value is set to __
            loss_test, accuracy_test, __, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) # loss is duplicated so the 2nd value is set to __

            for e in range(X.shape[0]):

                loss, accuracy, __, mse = model.evaluate(X[e].reshape([1, X.shape[1]]), Y[e].reshape([1, Y.shape[1]]), verbose=False)

                if e in train_indices:
                    condition = 'train'
                if e in test_indices:
                    condition = 'test'

                preds = model.predict(X, batch_size=cfg['batch_size']) 

                preds_binary = (preds > 0.5).astype(int)

                accuracies = preds_binary == Y

                f.write("{},{},{},{},{},{},{},{},{},{}\n".format(words[e],
                                                condition,
                                                epoch+1,
                                                loss,
                                                accuracy,
                                                mse,
                                                cfg['learning_rate'],
                                                cfg['batch_size'],
                                                seed,
                                                runtime))
f.close()

2024-02-12 11:45:24.548595: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-12 11:45:24.618837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-12 11:45:24.618944: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
with open('outputs/hu100_longitudinal_test.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{}\n".format("word",
                                            "condition",
                                             "epoch",
                                             "loss",
                                             "accuracy",
                                             "mse",
                                             "learning_rate",
                                             "batch_size",
                                             "seed",
                                             "time"))
    for seed in range(1): 
        model = learner(seed, cfg['learning_rate'], hidden)
                
        start_time = time.time()

        for epoch in range(1): # 

            model.fit(X[sample], Y[sample], epochs=iter_step, batch_size=cfg['batch_size'], verbose=False)

            end_time = time.time()
            runtime = end_time - start_time

            loss_train, accuracy_train, __, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) # loss is duplicated so the 2nd value is set to __
            loss_test, accuracy_test, __, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) # loss is duplicated so the 2nd value is set to __

            for e in range(X.shape[0]):

                loss, accuracy, __, mse = model.evaluate(X[e].reshape([1, X.shape[1]]), Y[e].reshape([1, Y.shape[1]]), verbose=False)

                if e in train_indices:
                    condition = 'train'
                if e in test_indices:
                    condition = 'test'

                preds = model.predict(X, batch_size=cfg['batch_size']) 

                preds_binary = (preds > 0.5).astype(int)

                accuracies = preds_binary == Y

                f.write("{},{},{},{},{},{},{},{},{},{}\n".format(words[e],
                                                condition,
                                                epoch+1,
                                                loss,
                                                accuracy,
                                                mse,
                                                cfg['learning_rate'],
                                                cfg['batch_size'],
                                                seed,
                                                runtime))
f.close()

In [45]:
preds = model.predict(X, batch_size=cfg['batch_size']) 

preds_binary = (preds > 0.5).astype(int)

accuracies = preds_binary == Y

#preds_binary = np.argmax(preds, axis=1)
#targets_binary = np.argmax(Y, axis=1)  # Assuming y_test is one-hot encoded



46/46 [==============================] - 0s 1ms/step


In [47]:
for e in 


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [44]:
words[0]

'fawn'

In [38]:
accuracies = np.all(accuracies, axis=1)

In [39]:
with open('data/hu100_longitudinal_test_alt.csv', 'w') as f:
    for word, accuracy in zip(words, accuracies):
        f.write('{},{}\n'.format(word, int(accuracy)))
f.close()

In [17]:
int(True)

1

How to grab sample from column in `samples`...

In [ ]:
for col in range(samples.shape[1]):
    sample = samples[:, col]
    sample = sample > 0

In [134]:


with open('outputs/tune_2.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{}\n".format("learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.19, .20, .21, .22, .23, .24, .25]: 
        for batch_size in [32, 48, 64, 80, 96]:
            for epochs in [100, 125, 150, 175, 200, 225]:
                model = learner(seed, learning_rate, hidden)
                
                start_time = time.time()


                model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=False)

                end_time = time.time()
                runtime = end_time - start_time

                loss_train, accuracy_train, __, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) # loss is duplicated so the 2nd value is set to __
                loss_test, accuracy_test, __, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) # loss is duplicated so the 2nd value is set to __

                f.write("{},{},{},{},{},{},{},{},{},{}\n".format(learning_rate,
                                                batch_size,
                                                epochs,
                                                loss_train,
                                                accuracy_train,
                                                mse_train,
                                                loss_test,
                                                accuracy_test,
                                                mse_test,
                                                runtime))
f.close()